## Восстановление параметров распределения денежных доходов населения по отчетным данным Росстата

Росстат при расчете уровня бедности моделирует распределение денежных доходов населения с помощью логнормального распределения ([1](https://rosstat.gov.ru/storage/mediabank/2v230419.pdf), [2](https://rosstat.gov.ru/storage/mediabank/metodika_bednost.docx)). В некоторых исследовательских задачах может пригодиться знание этих параметров. Например, для ответа на вопрос, какой уровень бедности был бы в 2022 году, если бы граница бедности была на 1000 рублей больше.

Несмотря на то, что Росстат не раскрывает параметры логнормального распределения, их [можно восстановить](https://voprstat.elpub.ru/jour/article/view/189) из публикуемых Росстатом данных. Ниже приведен код, который восстанавливает параметры распределения по опубликованным оценкам уровня бедности.

На вход нужно подать:

- значение среднедушевых денежных доходов (**average_income**) в рублях за соответствующий период (квартал, год);
- значение границы бедности (**poverty_line**) в рублях за соответствующий период (квартал, год);
- уровень бедности (**poverty**) в процентах (например, 9.8) за соответствующий период (квартал, год).

 Вместо уровня бедности можно использовать любые другие квинтили из распределения денежных доходов. Например, можно взять первый дециль распределения доходов (0.1). Тогда вместо значения границы бедности нужно указать верхнюю границу среднедушевых денежных доходов для этого дециля.

In [75]:
from scipy.optimize import fsolve
from scipy.stats import norm
from scipy.integrate import quad
import math
import numpy as np

In [76]:
class HackParams:
    def __init__(self, average_income, poverty_line, poverty):
        """Initialize class with parameters from Rosstat."""
        self.average_income = average_income
        self.poverty_line = poverty_line
        self.poverty = poverty/100

    def find_upper_limit(self):
        upper_limit, = fsolve(self.equation, 0, self.poverty)
        return upper_limit

    def find_params(self):
        """Find mean and standard deviation of lognormal distribution used by Rosstat."""
        u_limit = self.find_upper_limit()
        stdev = u_limit + math.sqrt(u_limit**2+2*math.log(self.average_income/self.poverty_line))
        mu = math.log(self.average_income) - 0.5*stdev**2
        return mu, stdev

    def count_poverty(self, new_line):
        """Count poverty based on some value of poverty line."""
        mu, sigma = self.find_params()
        ln_x0 = math.log(self.average_income) - 0.5 * sigma ** 2
        u = (math.log(new_line) - ln_x0) / sigma
        integral, error = quad(self.integrand, -np.inf, u)
        result = round((1 / math.sqrt(2 * math.pi)) * integral, 4)
        return result*100

    @staticmethod
    def equation(x, value):
        return norm.cdf(x) - value

    @staticmethod
    def integrand(x):
        return np.exp((-x ** 2) / 2)

### Пример
Оценим, какой уровень бедности был бы в 2022 году, если бы граница бедности составляла 14 500 рублей, а не 13 545 рублей (**poverty_line**). Для границы бедности 13 545 рублей уровень бедности (**poverty**), по данным Росстата, составил 9,8% в целом за 2022 год. Среднедушевые денежные доходы населения (**average_income**) за 2022 год составили 44 937 рублей.

In [77]:
new_poverty = HackParams(average_income=44937, poverty_line=13545, poverty=9.8)

In [78]:
new_poverty.find_params()

(10.450569448889533, 0.7244961545064188)

Это параметры (среднее и нормальное отклонение) логнормального распределения, используемого Росстатом для аппроксимации распределения среднедушевых денежных доходов населения в 2022 году. Посчитаем, какой бы уровень бедности был, если бы границу бедности установили на уровне 14 500 рублей.

In [79]:
new_poverty.count_poverty(14545)

11.61

Т.е. увеличение границы бедности на 1000 рублей, увеличило бы оценку уровня бедности на 1.8 процентных пункта, или примерно 2.6 млн человек.